## Section 1: Imports and loading cleaned data from Notebook 01

In [1]:
import pandas as pd
from pathlib import Path

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 120)

def load_folder(path):
    files = sorted(Path(path).glob("*.csv"))
    return pd.concat((pd.read_csv(f) for f in files), ignore_index=True)

df_enrolment = load_folder("../data/enrolment")
df_demo_updates = load_folder("../data/demographic")
df_bio_updates = load_folder("../data/biometric")


## Section 2: Date parsing and standardization

### Step 2.1 Convert to datetime

In [2]:
for df in [df_enrolment, df_demo_updates, df_bio_updates]:
    df["date"] = pd.to_datetime(df["date"], errors="coerce")

In [3]:
df_enrolment["date"].isna().mean()
df_demo_updates["date"].isna().mean()
df_bio_updates["date"].isna().mean()

0.5072784599281718

### Step 2.2 Create a standardized month column

In [4]:
for df in [df_enrolment, df_demo_updates, df_bio_updates]:
    df["year_month"] = df["date"].dt.to_period("M").astype(str)

## Section 3: Geographic field normalization

### Step 3.1 Clean text fields

In [8]:
for name, df in {
    "enrolment": df_enrolment,
    "demo": df_demo_updates,
    "bio": df_bio_updates,
}.items():

    # Force string dtype (memory-safe)
    df["state"] = df["state"].astype("string")
    df["district"] = df["district"].astype("string")

    # Remove numeric garbage states
    df = df.loc[~df["state"].str.match(r"^\d+$", na=False)].copy()

    # Normalize text
    df.loc[:, "state"] = df["state"].str.strip().str.upper()
    df.loc[:, "district"] = df["district"].str.strip().str.upper()

    # Assign back to original variable
    if name == "enrolment":
        df_enrolment = df
    elif name == "demo":
        df_demo_updates = df
    else:
        df_bio_updates = df


## Section 4: Age-group Alignment

The enrolment and update datasets use different age group definitions.

Enrolment data includes:
- age_0_5
- age_5_17
- age_18_greater

Demographic and biometric update data include:
- *_age_5_17
- *_age_17_ (17+)

To ensure valid comparisons, analysis is restricted to aligned age groups:
- Age 5–17
- Age 18+

The age_0_5 enrolment group is excluded from update ratio calculations, as updates for this group are not consistently recorded.


### Step 4.1 Create aligned enrolment columns

In [9]:
df_enrolment["enrol_age_5_17"] = df_enrolment["age_5_17"]
df_enrolment["enrol_age_18_plus"] = df_enrolment["age_18_greater"]

### Step 4.2 Rename update columns for consistency

In [10]:
df_demo_updates = df_demo_updates.rename(columns={
    "demo_age_5_17": "demo_age_5_17",
    "demo_age_17_": "demo_age_18_plus"
})

df_bio_updates = df_bio_updates.rename(columns={
    "bio_age_5_17": "bio_age_5_17",
    "bio_age_17_": "bio_age_18_plus"
})

## Section 5: Final aggregation

### Step 5.1 Enrolment aggregation

In [11]:
enrol_agg = (
    df_enrolment
    .groupby(["state", "year_month"], as_index=False)
    .agg({
        "enrol_age_5_17": "sum",
        "enrol_age_18_plus": "sum"
    })
)

### Step 5.2 Demographic update aggregation

In [12]:
demo_agg = (
    df_demo_updates
    .groupby(["state", "year_month"], as_index=False)
    .agg({
        "demo_age_5_17": "sum",
        "demo_age_18_plus": "sum"
    })
)

### Step 5.3 Biometric update aggregation

In [13]:
bio_agg = (
    df_bio_updates
    .groupby(["state", "year_month"], as_index=False)
    .agg({
        "bio_age_5_17": "sum",
        "bio_age_18_plus": "sum"
    })
)

## Section 6: Validation checks

In [14]:
enrol_agg.head()
demo_agg.head()
bio_agg.head()

enrol_agg.isna().sum()
demo_agg.isna().sum()
bio_agg.isna().sum()

state              0
year_month         0
bio_age_5_17       0
bio_age_18_plus    0
dtype: int64

## Section 7: Save clean outputs for next notebooks

In [15]:
enrol_agg.to_csv("../data/enrolment_clean_monthly.csv", index=False)
demo_agg.to_csv("../data/demographic_clean_monthly.csv", index=False)
bio_agg.to_csv("../data/biometric_clean_monthly.csv", index=False)